In [80]:
import pandas as pd
import numpy as np
import pingouin as pg
import numpy as np, statsmodels.stats.api as sms
import requests
from bs4 import BeautifulSoup 

with open('C:/Users/USER/Desktop/연구 결과물/청년가구/청년가구 gb AUC 부트스트랩 95% 신뢰 구간.html', 'r', encoding='utf-8') as f:
    html_content = f.read()

In [81]:
from selenium import webdriver

In [82]:
options = webdriver.ChromeOptions() 
options.add_argument('lang=ko_KR') 
options.add_argument('disable-gpu')
driver = webdriver.Chrome(options=options)

### XGBClassifier

In [83]:
driver.get(url='.html')

In [84]:
df = pd.DataFrame()

In [85]:
import warnings
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

In [86]:
for i in range(0, 780):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

df = df.dropna(axis=1, how='all')

In [87]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')).any()]
df = df.drop(columns=cols_to_drop)

In [88]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [89]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [90]:
df

,Column_31,Column_50,Column_67,Column_83,Column_99,Column_114,Column_130,Column_144,Column_159,Column_174,...,Column_657,Column_670,Column_683,Column_696,Column_709,Column_722,Column_735,Column_748,Column_761,Column_774
0,"0.7692385993485342, 0.7706622670104959, 0.7742...","0.7692159790083244, 0.7755072611292074, 0.7662...","0.7714447894046326, 0.7663742987694535, 0.7679...","0.7710736744480637, 0.7671052185124865, 0.7723...","0.7740086635903003, 0.7746752567408614, 0.7805...","0.770581682048498, 0.7679775153818313, 0.77158...","0.7758281872059355, 0.7652574194715889, 0.7735...","0.7750067861020631, 0.7750393028411147, 0.7771...","0.7700006220593557, 0.7689728103510678, 0.7705...","0.7756719654813609, 0.7668549809989142, 0.7721...",...,"0.7619456602877307, 0.7640677309536735, 0.7645...","0.7647371516467607, 0.7632265370521172, 0.7727...","0.7696379897303656, 0.7649167005971769, 0.7712...","0.7641037821208831, 0.769820366223308, 0.76844...","0.7702826694263482, 0.7647993575823381, 0.7671...","0.7613575314422729, 0.7600398966250452, 0.7593...","0.7580839440825191, 0.7521729664314152, 0.7549...","0.7525313574466158, 0.7495087144860658, 0.7550...","0.748012944489685, 0.7459445971317409, 0.74697...","0.7159259919019181, 0.7181781295240679, 0.7151..."


In [91]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'xgb_{i}' for i in range(len(df.columns))])

In [92]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data)

112000

In [93]:
pd.set_option('display.precision', 16)

In [94]:
new_df.shape

(2000, 56)

In [95]:
new_df = new_df.astype(float)

In [96]:
new_df

,xgb_0,xgb_1,xgb_2,xgb_3,xgb_4,xgb_5,xgb_6,xgb_7,xgb_8,xgb_9,...,xgb_46,xgb_47,xgb_48,xgb_49,xgb_50,xgb_51,xgb_52,xgb_53,xgb_54,xgb_55
0,0.7692385993485342,0.7692159790083244,0.7714447894046326,0.7710736744480637,0.7740086635903003,0.7705816820484980,0.7758281872059355,0.7750067861020631,0.7700006220593557,0.7756719654813609,...,0.7619456602877307,0.7647371516467607,0.7696379897303656,0.7641037821208831,0.7702826694263482,0.7613575314422729,0.7580839440825191,0.7525313574466158,0.7480129444896850,0.7159259919019181
1,0.7706622670104959,0.7755072611292074,0.7663742987694535,0.7671052185124865,0.7746752567408614,0.7679775153818313,0.7652574194715889,0.7750393028411147,0.7689728103510678,0.7668549809989142,...,0.7640677309536735,0.7632265370521172,0.7649167005971769,0.7698203662233080,0.7647993575823381,0.7600398966250452,0.7521729664314152,0.7495087144860658,0.7459445971317409,0.7181781295240679
2,0.7742546597900832,0.7662838174086140,0.7679365160152010,0.7723163793883461,0.7805318041983351,0.7715854596453131,0.7735096023344191,0.7771585459645313,0.7705576479370250,0.7721424855229821,...,0.7645003449601881,0.7727701999638075,0.7712949296507419,0.7684497149837134,0.7671949929876944,0.7593627001900110,0.7549658715617084,0.7550888696615998,0.7469752363825553,0.7151950721588852
3,0.7767902585504887,0.7687204521806007,0.7718781102967790,0.7690929809084329,0.7784054922186030,0.7730161961635904,0.7710948810170105,0.7635418080437929,0.7773621290264205,0.7675943833695259,...,0.7668563947701774,0.7687289348081794,0.7664895211273978,0.7667659134093376,0.7689982582338040,0.7624722900832429,0.7572802151194353,0.7509033998371335,0.7488477764205574,0.7206720220322113
4,0.7699384161237784,0.7698224868802026,0.7708828153275425,0.7668408432862830,0.7764841770720232,0.7723319308722403,0.7668267055736517,0.7761703198516107,0.7689021217879116,0.7734445688563155,...,0.7640740929243576,0.7653316424629028,0.7676728476746291,0.7677449500090482,0.7695432670557366,0.7607269894589215,0.7566277596815056,0.7552797287821208,0.7465221226927252,0.7211201875226203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.7738121493847268,0.7701759296959827,0.7659247195077814,0.7707527483713354,0.7691035841929063,0.7744455189106044,0.7706071299312343,0.7785991788816504,0.7679039992761491,0.7677456568946797,...,0.7647746165852335,0.7650898875769091,0.7613681347267462,0.7637178225660515,0.7655111914133188,0.7632010891693812,0.7592807014567499,0.7504545274610929,0.7502346860296779,0.7156107209102424
1996,0.7698634862468332,0.7682828899746652,0.7711104325009048,0.7646254637169743,0.7737343919652552,0.7748399610930148,0.7733837766920014,0.7728479573832792,0.7692032550669562,0.7755567431234165,...,0.7671370283659067,0.7630024543069127,0.7685529202859211,0.7696379897303656,0.7649972855591748,0.7637157019091567,0.7566143288545061,0.7573812997647484,0.7451048170014477,0.7160942306822294
1997,0.7653606247737965,0.7669101180781759,0.7674346272167933,0.7714511513753168,0.7679209645313065,0.7665043657256605,0.7716504931234165,0.7672423543250091,0.7703017553384004,0.7804215300398117,...,0.7684638526963445,0.7663714712269272,0.7672755779496923,0.7700861552207745,0.7677944320032573,0.7610168125678610,0.7562736099800941,0.7526084079804558,0.7449931290716613,0.7160942306822294
1998,0.7700585866811437,0.7783644928519724,0.7618671959826275,0.7710496403365907,0.7715826321027869,0.7730628506152732,0.7765421416938110,0.7711712246652190,0.7708912979551212,0.7637665976746291,...,0.7660173215255157,0.7607163861744480,0.7689311040988056,0.7698125904813609,0.7649895098172277,0.7626426495204488,0.7528536972946074,0.7510341736789721,0.7468614277958741,0.7206225400380021


In [97]:
driver.close()
driver.quit()

In [98]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")
    print()

AUC 신뢰 구간
xgb_0: [0.76328051 0.77836681]

xgb_1: [0.76368714 0.77841037]

xgb_2: [0.763322 0.77834399]

xgb_3: [0.76336265 0.7781291]

xgb_4: [0.76275603 0.77823223]

xgb_5: [0.76356167 0.77847056]

xgb_6: [0.76356381 0.77801815]

xgb_7: [0.7634543 0.77866898]

xgb_8: [0.76345281 0.77841981]

xgb_9: [0.76379487 0.77852139]

xgb_10: [0.76433479 0.77898093]

xgb_11: [0.76394424 0.77914645]

xgb_12: [0.76421377 0.77908226]

xgb_13: [0.76334441 0.77848922]

xgb_14: [0.76373369 0.77851548]

xgb_15: [0.7639037 0.7789359]

xgb_16: [0.76394424 0.77884768]

xgb_17: [0.76418783 0.77910329]

xgb_18: [0.76418769 0.7788643]

xgb_19: [0.7644187 0.77922792]

xgb_20: [0.76436183 0.77961731]

xgb_21: [0.76493529 0.77956786]

xgb_22: [0.76487356 0.77967573]

xgb_23: [0.76518772 0.77999199]

xgb_24: [0.76611667 0.78048002]

xgb_25: [0.76634302 0.7809542]

xgb_26: [0.7663397 0.7805464]

xgb_27: [0.76579539 0.78016864]

xgb_28: [0.76603418 0.78073194]

xgb_29: [0.76611682 0.78053767]

xgb_30: [0.76678587 0

In [99]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
xgb_0: [0.76 0.78]
xgb_1: [0.76 0.78]
xgb_2: [0.76 0.78]
xgb_3: [0.76 0.78]
xgb_4: [0.76 0.78]
xgb_5: [0.76 0.78]
xgb_6: [0.76 0.78]
xgb_7: [0.76 0.78]
xgb_8: [0.76 0.78]
xgb_9: [0.76 0.78]
xgb_10: [0.76 0.78]
xgb_11: [0.76 0.78]
xgb_12: [0.76 0.78]
xgb_13: [0.76 0.78]
xgb_14: [0.76 0.78]
xgb_15: [0.76 0.78]
xgb_16: [0.76 0.78]
xgb_17: [0.76 0.78]
xgb_18: [0.76 0.78]
xgb_19: [0.76 0.78]
xgb_20: [0.76 0.78]
xgb_21: [0.76 0.78]
xgb_22: [0.76 0.78]
xgb_23: [0.77 0.78]
xgb_24: [0.77 0.78]
xgb_25: [0.77 0.78]
xgb_26: [0.77 0.78]
xgb_27: [0.77 0.78]
xgb_28: [0.77 0.78]
xgb_29: [0.77 0.78]
xgb_30: [0.77 0.78]
xgb_31: [0.76 0.78]
xgb_32: [0.77 0.78]
xgb_33: [0.76 0.78]
xgb_34: [0.76 0.78]
xgb_35: [0.76 0.78]
xgb_36: [0.76 0.78]
xgb_37: [0.76 0.78]
xgb_38: [0.76 0.78]
xgb_39: [0.76 0.78]
xgb_40: [0.76 0.78]
xgb_41: [0.76 0.78]
xgb_42: [0.76 0.78]
xgb_43: [0.76 0.77]
xgb_44: [0.76 0.77]
xgb_45: [0.76 0.77]
xgb_46: [0.76 0.77]
xgb_47: [0.76 0.77]
xgb_48: [0.76 0.77]
xgb_49: [0.76 0.77]


In [100]:
import pingouin as pg
import numpy as np, statsmodels.stats.api as sms

In [101]:
i = 0
while i < 55:  
    base_column = f'xgb_{i}'
    continue_search = True  

    for j in range(i + 1, 56):
        if not continue_search:
            break  
        compare_column = f'xgb_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'xgb_55':
            break
    if compare_column == 'xgb_55' and continue_search:
        break
    i += 1  

xgb_0 and xgb_1:
(-0.0002293013976050299, 0.00024045605287071817)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1   5.577e-06      0.000      0.047      0.963      -0.000       0.000
-------------------------------------------
xgb_1 and xgb_2:
(-0.0002662990981285397, 0.00020083936798948943)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1  -3.273e-05      0.000     -0.275      0.784      -0.000       0.000
-------------------------------------------
xgb_2 and xgb_3:
(-0.00022436011132597776, 0.00025070008372917764)
                          Test for equality of means                          
  

In [102]:
## xgboost는 X = 30

In [103]:
del new_df

### LGBMClassifier

In [104]:
driver = webdriver.Chrome(options=options)
driver.get(url='.html')

In [105]:
df = pd.DataFrame()

In [106]:
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

In [107]:
for i in range(0, 775):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

df = df.dropna(axis=1, how='all')

In [108]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [109]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [110]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [111]:
df

,Column_31,Column_48,Column_62,Column_75,Column_88,Column_100,Column_113,Column_126,Column_139,Column_152,...,Column_632,Column_645,Column_658,Column_671,Column_684,Column_697,Column_710,Column_723,Column_736,Column_749
0,"0.763806890155628, 0.7719205234346724, 0.77541...","0.7663120928338761, 0.7756670172819399, 0.7683...","0.7636810645132104, 0.7639256469417299, 0.7656...","0.7635326185305827, 0.7646735319399205, 0.7626...","0.7694181482989504, 0.7739535265110388, 0.7743...","0.7672366992399566, 0.7719007306369887, 0.7664...","0.775292367897213, 0.7603078062794064, 0.77104...","0.7705873371335505, 0.7635382736156352, 0.7674...","0.764361088490771, 0.7634746539087949, 0.77082...","0.7749318562251176, 0.7643893639160333, 0.7711...",...,"0.7611016388436482, 0.7638280967245747, 0.7624...","0.7618049900470504, 0.7571727685034382, 0.7654...","0.7620361416485705, 0.7625026861654, 0.7683620...","0.7495836443630112, 0.7671964067589576, 0.7604...","0.7693729076185306, 0.7607397134002896, 0.7666...","0.7630667808993847, 0.7569267723036555, 0.7614...","0.7567387407256605, 0.750807970276873, 0.75361...","0.753233294878755, 0.7493807681867535, 0.75154...","0.7478270335685848, 0.7480214271172638, 0.7433...","0.7161748156442271, 0.7213237705845095, 0.7148..."


In [112]:
#pd.set_option('display.max_columns', None)

In [113]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'lgbm_{i}' for i in range(len(df.columns))])

In [114]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data) # 2000 * 56

112000

In [115]:
new_df.shape

(2000, 56)

In [116]:
new_df = new_df.astype(float)

In [117]:
new_df

,lgbm_0,lgbm_1,lgbm_2,lgbm_3,lgbm_4,lgbm_5,lgbm_6,lgbm_7,lgbm_8,lgbm_9,...,lgbm_46,lgbm_47,lgbm_48,lgbm_49,lgbm_50,lgbm_51,lgbm_52,lgbm_53,lgbm_54,lgbm_55
0,0.7638068901556280,0.7663120928338761,0.7636810645132104,0.7635326185305827,0.7694181482989504,0.7672366992399566,0.7752923678972130,0.7705873371335505,0.7643610884907710,0.7749318562251176,...,0.7611016388436482,0.7618049900470504,0.7620361416485705,0.7495836443630112,0.7693729076185306,0.7630667808993847,0.7567387407256605,0.7532332948787550,0.7478270335685848,0.7161748156442271
1,0.7719205234346724,0.7756670172819399,0.7639256469417299,0.7646735319399205,0.7739535265110388,0.7719007306369887,0.7603078062794064,0.7635382736156352,0.7634746539087949,0.7643893639160333,...,0.7638280967245747,0.7571727685034382,0.7625026861654000,0.7671964067589576,0.7607397134002896,0.7569267723036555,0.7508079702768730,0.7493807681867535,0.7480214271172638,0.7213237705845095
2,0.7754111246833153,0.7683125791711907,0.7656447927976837,0.7626645629750270,0.7743748303474485,0.7664803316141875,0.7710453990228013,0.7674925918385813,0.7708276782482808,0.7711768797502714,...,0.7624122048045603,0.7654051585685849,0.7683620611653998,0.7604682693177706,0.7666655356496562,0.7614084272077452,0.7536178406623236,0.7515452519905897,0.7433086206116539,0.7148939388798407
3,0.7662145426167210,0.7639737151646760,0.7672706297502714,0.7631961409699601,0.7727277868259139,0.7735746358125226,0.7669030492218604,0.7646056709192907,0.7710821570756424,0.7675802456568946,...,0.7664152981360839,0.7623302060712993,0.7596433337857401,0.7568659801393413,0.7664054017372421,0.7560671993756787,0.7534291021986971,0.7542787787278321,0.7485678497104596,0.7213068053293521
4,0.7659473398479913,0.7704205121245024,0.7654991743575823,0.7628525945530221,0.7681287889069851,0.7681287889069851,0.7682786486608759,0.7710736744480637,0.7655726904632646,0.7687720548317046,...,0.7602972029949331,0.7643080720684039,0.7615512181053203,0.7570907697701773,0.7660759930329353,0.7604315112649295,0.7484794890065147,0.7551772303655444,0.7474340051574375,0.7206960561436844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.7761745611653998,0.7702423769453492,0.7592107197792254,0.7647244277053926,0.7653889001990590,0.7684115431596092,0.7651047321751719,0.7776194353963084,0.7620835029858849,0.7569260654180239,...,0.7621386400651465,0.7587901228284473,0.7583433711093015,0.7546399972855591,0.7665213309808181,0.7567210685848715,0.7595768865363736,0.7513770132102787,0.7505308711093015,0.7162822622602244
1996,0.7704714078899748,0.7700501040535649,0.7676297276511039,0.7580259794607309,0.7734841544516829,0.7698196593376765,0.7691990137531669,0.7736905650560985,0.7686236088490771,0.7745826547231270,...,0.7665898988870793,0.7648848907437568,0.7669539449873326,0.7584741449511401,0.7627776646760768,0.7636747025425263,0.7544894306460370,0.7580316345457835,0.7481946140969961,0.7161041270810712
1997,0.7609355207202317,0.7642168838219328,0.7678135179153095,0.7636117897213173,0.7618629546688382,0.7604619073470865,0.7660279248099892,0.7597974348534202,0.7667956026058631,0.7707032663771264,...,0.7698705551031487,0.7605870261038726,0.7616671473488962,0.7613858068675353,0.7678573448244662,0.7597776420557365,0.7563499536283025,0.7549375961364460,0.7459438902461093,0.7149674549855229
1998,0.7639284744842563,0.7723587925262396,0.7619619186572566,0.7645519476112920,0.7711698108939560,0.7745911373507057,0.7744412775968150,0.7695524565689468,0.7680453764024611,0.7604534247195078,...,0.7582267349800942,0.7549736473036555,0.7621838807455665,0.7600547412233079,0.7619513153727832,0.7609475377759680,0.7501639974665220,0.7530551596996019,0.7463086432319943,0.7212219790535649


In [118]:
driver.close()
driver.quit()

In [119]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
lgbm_0: [0.75952168 0.77729169]
lgbm_1: [0.75884544 0.77605213]
lgbm_2: [0.75810646 0.7761635]
lgbm_3: [0.75854042 0.77602781]
lgbm_4: [0.75843279 0.77596027]
lgbm_5: [0.75947782 0.77641716]
lgbm_6: [0.75970102 0.77594235]
lgbm_7: [0.75802541 0.77589234]
lgbm_8: [0.75916657 0.77662456]
lgbm_9: [0.75865277 0.77597278]
lgbm_10: [0.76206707 0.77739853]
lgbm_11: [0.76034184 0.77781609]
lgbm_12: [0.76218812 0.77948052]
lgbm_13: [0.75723724 0.77633803]
lgbm_14: [0.76110719 0.776733]
lgbm_15: [0.75667042 0.77564602]
lgbm_16: [0.75994425 0.77794195]
lgbm_17: [0.75723939 0.77515679]
lgbm_18: [0.75867766 0.77679333]
lgbm_19: [0.76109934 0.77783037]
lgbm_20: [0.75943975 0.777422]
lgbm_21: [0.75696813 0.77646484]
lgbm_22: [0.76200815 0.77979675]
lgbm_23: [0.7601272 0.77694796]
lgbm_24: [0.75881119 0.77658862]
lgbm_25: [0.75630227 0.77399802]
lgbm_26: [0.75960792 0.77671073]
lgbm_27: [0.76046873 0.77811956]
lgbm_28: [0.76355368 0.78001185]
lgbm_29: [0.76549348 0.77983428]
lgbm_30: [0.7659

In [120]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
lgbm_0: [0.76 0.78]
lgbm_1: [0.76 0.78]
lgbm_2: [0.76 0.78]
lgbm_3: [0.76 0.78]
lgbm_4: [0.76 0.78]
lgbm_5: [0.76 0.78]
lgbm_6: [0.76 0.78]
lgbm_7: [0.76 0.78]
lgbm_8: [0.76 0.78]
lgbm_9: [0.76 0.78]
lgbm_10: [0.76 0.78]
lgbm_11: [0.76 0.78]
lgbm_12: [0.76 0.78]
lgbm_13: [0.76 0.78]
lgbm_14: [0.76 0.78]
lgbm_15: [0.76 0.78]
lgbm_16: [0.76 0.78]
lgbm_17: [0.76 0.78]
lgbm_18: [0.76 0.78]
lgbm_19: [0.76 0.78]
lgbm_20: [0.76 0.78]
lgbm_21: [0.76 0.78]
lgbm_22: [0.76 0.78]
lgbm_23: [0.76 0.78]
lgbm_24: [0.76 0.78]
lgbm_25: [0.76 0.77]
lgbm_26: [0.76 0.78]
lgbm_27: [0.76 0.78]
lgbm_28: [0.76 0.78]
lgbm_29: [0.77 0.78]
lgbm_30: [0.77 0.78]
lgbm_31: [0.76 0.78]
lgbm_32: [0.76 0.78]
lgbm_33: [0.76 0.77]
lgbm_34: [0.76 0.78]
lgbm_35: [0.77 0.78]
lgbm_36: [0.76 0.78]
lgbm_37: [0.76 0.77]
lgbm_38: [0.76 0.78]
lgbm_39: [0.76 0.78]
lgbm_40: [0.76 0.77]
lgbm_41: [0.76 0.78]
lgbm_42: [0.76 0.78]
lgbm_43: [0.75 0.77]
lgbm_44: [0.76 0.78]
lgbm_45: [0.76 0.77]
lgbm_46: [0.76 0.77]
lgbm_47: [0.7

In [121]:
i = 0
while i < 55:  
    base_column = f'lgbm_{i}'
    continue_search = True  

    for j in range(i + 1, 56):
        if not continue_search:
            break  
        compare_column = f'lgbm_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'lgbm_55':
            break
    if compare_column == 'lgbm_55' and continue_search:
        break
    i += 1  

lgbm_0 and lgbm_1:
(0.0011206080215228269, 0.0016720511125706068)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0014      0.000      9.929      0.000       0.001       0.002
-------------------------------------------
lgbm_0 and lgbm_2:
(0.00109889980323245, 0.0016607067725002594)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0014      0.000      9.630      0.000       0.001       0.002
-------------------------------------------
lgbm_0 and lgbm_3:
(0.0009426870597779047, 0.0015036772860417802)
                          Test for equality of means                          
     

In [122]:
### lgbm = 30

In [123]:
del new_df

### GradientBoosting

In [124]:
driver = webdriver.Chrome(options=options)
driver.get(url='.html')

In [125]:
df = pd.DataFrame()

In [126]:
for i in range(0, 775):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

df = df.dropna(axis=1, how='all')

In [127]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [128]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [129]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [130]:
df

,Column_28,Column_41,Column_53,Column_65,Column_77,Column_88,Column_100,Column_112,Column_124,Column_136,...,Column_583,Column_595,Column_608,Column_621,Column_634,Column_647,Column_660,Column_673,Column_686,Column_699
0,"0.7672338716974303, 0.7767146217879117, 0.7743...","0.7682362355229824, 0.7689233283568585, 0.7744...","0.7694761129207384, 0.768941707383279, 0.76825...","0.7729115770901194, 0.7641688155989868, 0.7689...","0.7717904564784653, 0.7826157030401738, 0.7755...","0.7701009998190371, 0.7733767078356858, 0.7735...","0.7783121833152371, 0.7682489594643503, 0.7720...","0.7677711047774158, 0.7740821796959827, 0.7762...","0.7708969530401737, 0.7686617806731812, 0.7693...","0.7726019611834962, 0.7697249366630473, 0.7693...",...,----------------------------------------------...,"0.7556176201140066, 0.7627649407347087, 0.7629...","0.7644911554469778, 0.7592411158613824, 0.7637...","0.759702712178791, 0.7592495984889612, 0.76082...","0.7696577825280492, 0.759168306641332, 0.76861...","0.7600144487423092, 0.7483465945077814, 0.7633...","0.7635686696977924, 0.7549503200778142, 0.7530...","0.7497688483984799, 0.7464245724755698, 0.7450...","0.7487855704849801, 0.749940621606949, 0.74620...","0.7201149961545421, 0.721130083921462, 0.71610..."


In [132]:
df = df.drop('Column_583',axis=1)

In [133]:
df.shape

(1, 56)

In [134]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'gb_{i}' for i in range(len(df.columns))])

In [135]:
new_df.shape

(2000, 56)

In [136]:
new_df = new_df.astype(float)

In [137]:
new_df

,gb_0,gb_1,gb_2,gb_3,gb_4,gb_5,gb_6,gb_7,gb_8,gb_9,...,gb_46,gb_47,gb_48,gb_49,gb_50,gb_51,gb_52,gb_53,gb_54,gb_55
0,0.7672338716974303,0.7682362355229824,0.7694761129207384,0.7729115770901194,0.7717904564784653,0.7701009998190371,0.7783121833152371,0.7677711047774158,0.7708969530401737,0.7726019611834962,...,0.7652503506152732,0.7556176201140066,0.7644911554469778,0.7597027121787910,0.7696577825280492,0.7600144487423092,0.7635686696977924,0.7497688483984799,0.7487855704849801,0.7201149961545421
1,0.7767146217879117,0.7689233283568585,0.7689417073832790,0.7641688155989868,0.7826157030401738,0.7733767078356858,0.7682489594643503,0.7740821796959827,0.7686617806731812,0.7697249366630473,...,0.7616735093195802,0.7627649407347087,0.7592411158613824,0.7592495984889612,0.7591683066413319,0.7483465945077814,0.7549503200778142,0.7464245724755698,0.7499406216069490,0.7211300839214621
2,0.7743776578899746,0.7744681392508144,0.7682532007781396,0.7689445349258053,0.7755581568946797,0.7735760495837857,0.7720194874230909,0.7762424221860296,0.7693898728736880,0.7693432184220050,...,0.7565012271534564,0.7629473172276511,0.7637708389884184,0.7608287809898661,0.7686130055646038,0.7633028807003258,0.7530586941277597,0.7450829035468693,0.7462047310441550,0.7161069546235974
3,0.7755213988418386,0.7705661305646037,0.7737442883640969,0.7667899475208109,0.7778795693087224,0.7756938789359392,0.7776830551031488,0.7663332994028229,0.7779403614730366,0.7646198086319218,...,0.7644897416757147,0.7640755066956206,0.7592093060079623,0.7609206761219689,0.7674035242490047,0.7445506186663047,0.7583617501357220,0.7433559819489685,0.7487650708016649,0.7212983227017733
4,0.7677880700325732,0.7711599144951139,0.7716391829533116,0.7732452271082157,0.7749770969055374,0.7714129795512124,0.7640118869887803,0.7765110387260223,0.7686066435939197,0.7747933066413319,...,0.7644409665671372,0.7633947758324284,0.7609758132012305,0.7635863418385812,0.7670288748642780,0.7491793057817590,0.7572399226384365,0.7500841193901557,0.7437956648117988,0.7201885122602243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.7701080686753530,0.7712376719145857,0.7703272032211365,0.7622842585052479,0.7657663771263120,0.7711669833514295,0.7735477741585233,0.7792989956568946,0.7709662278320666,0.7672282166123778,...,0.7597691594281578,0.7613398593014840,0.7593365454216432,0.7632972256152731,0.7676912267010495,0.7541147812613102,0.7591004456207020,0.7473498857672820,0.7493616822747013,0.7209887067951500
1996,0.7652701434129570,0.7679859980094099,0.7696372828447339,0.7692513232899023,0.7775261264929424,0.7767330008143322,0.7704431324647123,0.7792707202316322,0.7734389137712631,0.7784351814151285,...,0.7669087043069128,0.7647703752714441,0.7625698403003982,0.7649633550488599,0.7640479381559899,0.7524239108306189,0.7532446050488599,0.7511840334328628,0.7421641727741586,0.7160334385179152
1997,0.7646763594824465,0.7650821118349620,0.7660491313789358,0.7707060939196525,0.7716165626131016,0.7627776646760768,0.7727405107672819,0.7682093738689829,0.7737315644227289,0.7738856654904090,...,0.7630144713626493,0.7614543747737965,0.7648431844914949,0.7645031725027143,0.7711705177795873,0.7551440067408614,0.7623634296959826,0.7501915660061527,0.7453741404270720,0.7155866867987694
1998,0.7733258120702137,0.7762084916757148,0.7686038160513935,0.7668436708288093,0.7727263730546508,0.7721863124321391,0.7726811323742310,0.7750223375859573,0.7735392915309447,0.7723305171009772,...,0.7638959577452045,0.7635672559265291,0.7599409326366269,0.7629529723127036,0.7643144340390879,0.7538687850615273,0.7492754422276511,0.7467603431505609,0.7442523129297864,0.7151597278773072


In [138]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data) # 2000 * 56

112000

In [139]:
driver.close()
driver.quit()

In [140]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
gb_0: [0.76 0.78]
gb_1: [0.76 0.78]
gb_2: [0.76 0.78]
gb_3: [0.76 0.78]
gb_4: [0.76 0.78]
gb_5: [0.76 0.78]
gb_6: [0.76 0.78]
gb_7: [0.76 0.78]
gb_8: [0.76 0.78]
gb_9: [0.76 0.78]
gb_10: [0.76 0.78]
gb_11: [0.76 0.78]
gb_12: [0.76 0.78]
gb_13: [0.76 0.78]
gb_14: [0.76 0.78]
gb_15: [0.76 0.78]
gb_16: [0.76 0.78]
gb_17: [0.76 0.78]
gb_18: [0.76 0.78]
gb_19: [0.76 0.78]
gb_20: [0.76 0.78]
gb_21: [0.76 0.78]
gb_22: [0.77 0.78]
gb_23: [0.76 0.78]
gb_24: [0.76 0.78]
gb_25: [0.77 0.78]
gb_26: [0.76 0.78]
gb_27: [0.77 0.78]
gb_28: [0.77 0.78]
gb_29: [0.77 0.78]
gb_30: [0.76 0.78]
gb_31: [0.77 0.78]
gb_32: [0.76 0.78]
gb_33: [0.76 0.78]
gb_34: [0.76 0.78]
gb_35: [0.76 0.78]
gb_36: [0.76 0.78]
gb_37: [0.76 0.78]
gb_38: [0.76 0.78]
gb_39: [0.76 0.78]
gb_40: [0.76 0.78]
gb_41: [0.76 0.78]
gb_42: [0.76 0.78]
gb_43: [0.76 0.78]
gb_44: [0.75 0.77]
gb_45: [0.75 0.77]
gb_46: [0.75 0.77]
gb_47: [0.75 0.77]
gb_48: [0.75 0.77]
gb_49: [0.75 0.77]
gb_50: [0.76 0.77]
gb_51: [0.74 0.76]
gb_52: [0.75

In [141]:
i = 0
while i < 55:  
    base_column = f'gb_{i}'
    continue_search = True  

    for j in range(i + 1, 56):
        if not continue_search:
            break  
        compare_column = f'gb_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'gb_55':
            break
    if compare_column == 'gb_55' and continue_search:
        break
    i += 1  

gb_0 and gb_1:
(-8.54817184619297e-05, 0.0004246765474520711)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0002      0.000      1.304      0.192   -8.55e-05       0.000
-------------------------------------------
gb_1 and gb_2:
(-0.0003760607849092982, 0.0001382390105695678)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1     -0.0001      0.000     -0.907      0.365      -0.000       0.000
-------------------------------------------
gb_2 and gb_3:
(-0.000299697518544578, 0.00021829681052781244)
                          Test for equality of means                          
             

In [143]:
## gb = 25

In [144]:
del new_df

### ExtraTrees

In [162]:
driver = webdriver.Chrome(options=options)
driver.get(url='.html')

In [163]:
df = pd.DataFrame()

In [164]:
for i in range(0, 775):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

In [165]:
df = df.dropna(axis=1, how='all')

In [166]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [167]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [168]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [169]:
#pd.set_option('display.max_columns', None)

In [170]:
df

,Column_22,Column_34,Column_45,Column_57,Column_69,Column_81,Column_93,Column_105,Column_117,Column_129,...,Column_591,Column_604,Column_617,Column_630,Column_643,Column_656,Column_669,Column_682,Column_695,Column_708
0,"0.7705746131921825, 0.7694322860115818, 0.7702...","0.771183948606587, 0.7668592223127035, 0.77352...","0.7666966386174447, 0.7693531148208469, 0.7674...","0.7669581863011219, 0.7627295964531307, 0.7650...","0.7638125452406804, 0.7700656555374593, 0.7666...","0.7637715458740499, 0.7662414042707202, 0.7757...","0.7632936911871154, 0.7625443924176619, 0.7674...","0.7724422050307636, 0.77189224800941, 0.765824...","0.7698168317951501, 0.7650029406442272, 0.7616...","0.7717126990589938, 0.7712136378031125, 0.7701...",...,"0.7624793589395584, 0.7713472391874774, 0.7697...","0.768533834373869, 0.765289229325009, 0.769375...","0.7637305465074194, 0.7618459894136808, 0.7699...","0.7664789178429243, 0.7683846815056099, 0.7677...","0.765949460504886, 0.7612670500814331, 0.76715...","0.7666591736789722, 0.758185028727832, 0.76595...","0.7566694659337676, 0.7527547333061889, 0.7538...","0.752604873552298, 0.7491072034473399, 0.75224...","0.7453783817408615, 0.7462549199239956, 0.7458...","0.7158524757962359, 0.7213152879569308, 0.7091..."


In [172]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'ext_{i}' for i in range(len(df.columns))])

In [173]:
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data)

112000

In [174]:
new_df.shape

(2000, 56)

In [175]:
new_df = new_df.astype(float)

In [176]:
new_df

,ext_0,ext_1,ext_2,ext_3,ext_4,ext_5,ext_6,ext_7,ext_8,ext_9,...,ext_46,ext_47,ext_48,ext_49,ext_50,ext_51,ext_52,ext_53,ext_54,ext_55
0,0.7705746131921825,0.7711839486065870,0.7666966386174447,0.7669581863011219,0.7638125452406804,0.7637715458740499,0.7632936911871154,0.7724422050307636,0.7698168317951501,0.7717126990589938,...,0.7624793589395584,0.7685338343738690,0.7637305465074194,0.7664789178429243,0.7659494605048860,0.7666591736789722,0.7566694659337676,0.7526048735522980,0.7453783817408615,0.7158524757962359
1,0.7694322860115818,0.7668592223127035,0.7693531148208469,0.7627295964531307,0.7700656555374593,0.7662414042707202,0.7625443924176619,0.7718922480094100,0.7650029406442272,0.7712136378031125,...,0.7713472391874774,0.7652892293250090,0.7618459894136808,0.7683846815056099,0.7612670500814331,0.7581850287278320,0.7527547333061889,0.7491072034473399,0.7462549199239956,0.7213152879569308
2,0.7702042051212450,0.7735237400470503,0.7674289721317409,0.7650750429786464,0.7666075710278680,0.7757249819037279,0.7674346272167933,0.7658243417480999,0.7616494752081072,0.7701943087224032,...,0.7697850219417300,0.7693750282754253,0.7699263990680419,0.7677795874049946,0.7671582349348535,0.7659579431324646,0.7538122342110025,0.7522415343376764,0.7458562364277959,0.7091427173814694
3,0.7709266422366992,0.7712404994571118,0.7665481926348172,0.7653153840933768,0.7746561708288091,0.7724591702859210,0.7669044629931234,0.7649873891603330,0.7683337857401376,0.7694577338943177,...,0.7637383222493666,0.7699631571208831,0.7670514952044880,0.7718851791530945,0.7639913873054651,0.7573367659699601,0.7519630213988417,0.7459961997828447,0.7486993304379298,0.7206960561436844
4,0.7600603963083605,0.7666584667933405,0.7669426348172277,0.7639383708830982,0.7684864730365544,0.7685712993123416,0.7637121674809989,0.7719516264024611,0.7650029406442272,0.7672678022077453,...,0.7686412809898661,0.7693142361111112,0.7625048068222945,0.7666316051393414,0.7694768198063699,0.7619301088038363,0.7484653512938835,0.7538355614368439,0.7475061074918568,0.7201955811165399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.7661184061708288,0.7669016354505971,0.7700543453673543,0.7648629772891784,0.7698380383640970,0.7671730795331162,0.7682461319218241,0.7683253031125590,0.7662300941006153,0.7690830845095911,...,0.7684073018458197,0.7686384534473398,0.7630590051574374,0.7624807727108217,0.7617689388798408,0.7638401137803114,0.7570766320575462,0.7533258968964893,0.7496734188382193,0.7162794347176981
1996,0.7665170896670286,0.7702748936844011,0.7704502013210279,0.7672960776330077,0.7614572023163229,0.7660307523525155,0.7660024769272531,0.7680439626311980,0.7717720774520449,0.7678078628302570,...,0.7663262305465073,0.7631063664947521,0.7719502126311979,0.7706028886174447,0.7633156046416938,0.7628646116087585,0.7539564388798408,0.7552655910694897,0.7419386762576909,0.7163020550579080
1997,0.7669709102424900,0.7656037934310532,0.7741019724936663,0.7607418340571841,0.7694676302931597,0.7640203696163590,0.7668648773977560,0.7707640585414404,0.7677795874049946,0.7708107129931234,...,0.7642084011943540,0.7704254603239232,0.7722039845729279,0.7696372828447341,0.7594729743485342,0.7612458435124864,0.7576308303926891,0.7524253246018819,0.7460470955483171,0.7149844202406803
1998,0.7677739323199421,0.7731151601520087,0.7601522914404634,0.7656348963988419,0.7654044516829532,0.7674911780673181,0.7753418498914224,0.7678842064784654,0.7705731994209193,0.7663460233441912,...,0.7695821457654721,0.7673688868530583,0.7683493372240318,0.7666740182772348,0.7648679254885994,0.7656257068856316,0.7491863746380746,0.7506906272620339,0.7432407595910242,0.7212248065960911


In [177]:
driver.close()
driver.quit()

In [178]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
ext_0: [0.76255973 0.77444481]
ext_1: [0.76464335 0.77548075]
ext_2: [0.76284507 0.77529449]
ext_3: [0.76085137 0.77310919]
ext_4: [0.76224665 0.77462415]
ext_5: [0.76109821 0.77395084]
ext_6: [0.76236184 0.77466914]
ext_7: [0.76190491 0.77466868]
ext_8: [0.76157688 0.77429368]
ext_9: [0.76145243 0.77407147]
ext_10: [0.76303373 0.77504786]
ext_11: [0.76283669 0.77501279]
ext_12: [0.76160508 0.77353441]
ext_13: [0.76216942 0.77475358]
ext_14: [0.7616212 0.77435984]
ext_15: [0.76271253 0.7748617]
ext_16: [0.7632903 0.77554554]
ext_17: [0.76124574 0.77414456]
ext_18: [0.76183701 0.77341089]
ext_19: [0.7602679 0.77341659]
ext_20: [0.7624701 0.77404867]
ext_21: [0.76073158 0.77276288]
ext_22: [0.76170436 0.77386319]
ext_23: [0.76193668 0.77458432]
ext_24: [0.76133099 0.77408402]
ext_25: [0.76295258 0.77465681]
ext_26: [0.76255144 0.77448958]
ext_27: [0.76181923 0.77460167]
ext_28: [0.76188268 0.77464645]
ext_29: [0.76350254 0.77509038]
ext_30: [0.76230511 0.77474888]
ext_31: [0.76

In [179]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
ext_0: [0.76 0.77]
ext_1: [0.76 0.78]
ext_2: [0.76 0.78]
ext_3: [0.76 0.77]
ext_4: [0.76 0.77]
ext_5: [0.76 0.77]
ext_6: [0.76 0.77]
ext_7: [0.76 0.77]
ext_8: [0.76 0.77]
ext_9: [0.76 0.77]
ext_10: [0.76 0.78]
ext_11: [0.76 0.78]
ext_12: [0.76 0.77]
ext_13: [0.76 0.77]
ext_14: [0.76 0.77]
ext_15: [0.76 0.77]
ext_16: [0.76 0.78]
ext_17: [0.76 0.77]
ext_18: [0.76 0.77]
ext_19: [0.76 0.77]
ext_20: [0.76 0.77]
ext_21: [0.76 0.77]
ext_22: [0.76 0.77]
ext_23: [0.76 0.77]
ext_24: [0.76 0.77]
ext_25: [0.76 0.77]
ext_26: [0.76 0.77]
ext_27: [0.76 0.77]
ext_28: [0.76 0.77]
ext_29: [0.76 0.78]
ext_30: [0.76 0.77]
ext_31: [0.76 0.77]
ext_32: [0.76 0.77]
ext_33: [0.76 0.77]
ext_34: [0.76 0.77]
ext_35: [0.76 0.77]
ext_36: [0.76 0.77]
ext_37: [0.76 0.77]
ext_38: [0.76 0.77]
ext_39: [0.76 0.77]
ext_40: [0.76 0.77]
ext_41: [0.76 0.77]
ext_42: [0.76 0.77]
ext_43: [0.76 0.77]
ext_44: [0.76 0.77]
ext_45: [0.76 0.77]
ext_46: [0.76 0.77]
ext_47: [0.76 0.77]
ext_48: [0.76 0.77]
ext_49: [0.76 0.77]


In [180]:
i = 0
while i < 55:  
    base_column = f'ext_{i}'
    continue_search = True  

    for j in range(i + 1, 56):
        if not continue_search:
            break  
        compare_column = f'ext_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'ext_55':
            break
    if compare_column == 'ext_55' and continue_search:
        break
    i += 1  

ext_0 and ext_1:
(-0.0017332384216585876, -0.0013756537754550453)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1     -0.0016   9.12e-05    -17.045      0.000      -0.002      -0.001
-------------------------------------------
ext_1 and ext_2:
(0.0009129720188683535, 0.0012801024812220771)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0011   9.36e-05     11.712      0.000       0.001       0.001
-------------------------------------------
ext_1 and ext_3:
(0.0028404254380029357, 0.0032077947653535556)
                          Test for equality of means                          
       

In [181]:
### ext = 1

In [182]:
del new_df

### rf

In [183]:
driver = webdriver.Chrome(options=options)
driver.get(url='.html')

In [184]:
df = pd.DataFrame()

In [185]:
for i in range(0, 775):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

df = df.dropna(axis=1, how='all')

In [186]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [187]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [188]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [189]:
df

,Column_22,Column_34,Column_46,Column_58,Column_70,Column_82,Column_94,Column_106,Column_118,Column_130,...,Column_582,Column_595,Column_608,Column_621,Column_634,Column_647,Column_660,Column_673,Column_686,Column_699
0,"0.77359160106768, 0.7733950868621065, 0.773568...","0.7686151262214984, 0.7738064942996743, 0.7751...","0.7649322520810713, 0.7702961002533478, 0.7734...","0.7708220231632283, 0.7651824895946435, 0.7677...","0.7726146851248642, 0.7695171122873689, 0.7699...","0.7721764160332971, 0.7770822023163229, 0.7739...","0.7725185486789722, 0.7660830618892509, 0.7710...","0.7752414721317409, 0.7701405854144047, 0.7714...","0.770604302388708, 0.7690378438291712, 0.76718...","0.7787207632102786, 0.7659982356134636, 0.7755...",...,"0.7658766512848354, 0.7708644363011221, 0.7754...","0.7728345265562794, 0.7734120521172638, 0.7734...","0.7683868021625045, 0.76730173271806, 0.771156...","0.7676968817861021, 0.7686646082157075, 0.7707...","0.7646608079985523, 0.7610295365092291, 0.7614...","0.7548386321480275, 0.7548315632917119, 0.7552...","0.7544088456840391, 0.7487071061798768, 0.7516...","0.75271444082519, 0.7490110670014477, 0.751682...","0.745220039359392, 0.7455409654361203, 0.74557...","0.7163529508233804, 0.7213152879569308, 0.7198..."


In [190]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'rf_{i}' for i in range(len(df.columns))])

In [191]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data) # 2000 * 56

112000

In [192]:
new_df.shape

(2000, 56)

In [193]:
new_df = new_df.astype(float)

In [194]:
new_df

,rf_0,rf_1,rf_2,rf_3,rf_4,rf_5,rf_6,rf_7,rf_8,rf_9,...,rf_46,rf_47,rf_48,rf_49,rf_50,rf_51,rf_52,rf_53,rf_54,rf_55
0,0.7735916010676800,0.7686151262214984,0.7649322520810713,0.7708220231632283,0.7726146851248642,0.7721764160332971,0.7725185486789722,0.7752414721317409,0.7706043023887080,0.7787207632102786,...,0.7658766512848354,0.7728345265562794,0.7683868021625045,0.7676968817861021,0.7646608079985523,0.7548386321480275,0.7544088456840391,0.7527144408251900,0.7452200393593920,0.7163529508233804
1,0.7733950868621065,0.7738064942996743,0.7702961002533478,0.7651824895946435,0.7695171122873689,0.7770822023163229,0.7660830618892509,0.7701405854144047,0.7690378438291712,0.7659982356134636,...,0.7708644363011221,0.7734120521172638,0.7673017327180600,0.7686646082157075,0.7610295365092291,0.7548315632917119,0.7487071061798768,0.7490110670014477,0.7455409654361203,0.7213152879569308
2,0.7735689807274702,0.7751623009410062,0.7734417413137893,0.7677159676981542,0.7699666915490408,0.7739521127397757,0.7710538816503801,0.7714695304017374,0.7671843897032211,0.7755440191820485,...,0.7754040558269997,0.7734367931143684,0.7711563800669563,0.7707315418023886,0.7614416508324287,0.7552465051574375,0.7516102854686935,0.7516823878031126,0.7455763097176982,0.7198562760133912
3,0.7715614255338400,0.7694025968150562,0.7744879320484981,0.7651259387441187,0.7799493304379299,0.7722315531125589,0.7716999751176259,0.7709747104596453,0.7702791349981903,0.7700840345638797,...,0.7711358803836410,0.7737315644227289,0.7711641558089033,0.7675851938563156,0.7592693912866449,0.7546880655085052,0.7503668736427795,0.7441590040264204,0.7491220480456025,0.7206225400380021
4,0.7669709102424901,0.7711457767824829,0.7672819399203764,0.7676127623959463,0.7762028365906622,0.7738969756605139,0.7670628053745927,0.7779658093557726,0.7665368824647122,0.7764149022801302,...,0.7738785966340933,0.7724938076818675,0.7688710188201231,0.7676078141965255,0.7620156419652552,0.7534545500814334,0.7505655085052481,0.7535061527325370,0.7440254026420559,0.7201814434039087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.7732014001990590,0.7690901533659066,0.7716193901556279,0.7657239639884184,0.7746533432862831,0.7721481406080347,0.7726033749547592,0.7716165626131017,0.7729144046326457,0.7711797072927977,...,0.7689636208378574,0.7716391829533117,0.7656518616539992,0.7642409179334057,0.7616056482989505,0.7538666644046327,0.7561732322204126,0.7520195722493666,0.7505782324466160,0.7161182647937024
1996,0.7685854370249730,0.7720138323380383,0.7752782301845820,0.7707089214621787,0.7703187205935578,0.7745699307817591,0.7725454103329715,0.7751806799674268,0.7769083084509592,0.7783404587404994,...,0.7735378777596815,0.7715324432229460,0.7709499694625408,0.7718816447249367,0.7650460606677525,0.7515954408704307,0.7507189026872965,0.7564793136988779,0.7429325574556642,0.7161012995385450
1997,0.7693432184220050,0.7662442318132464,0.7723616200687659,0.7686419878754976,0.7752089553926892,0.7709860206297503,0.7728013029315961,0.7717183541440463,0.7744469326818676,0.7730741607853784,...,0.7675201603782121,0.7741472131740861,0.7707145765472312,0.7700126391150923,0.7623266716431415,0.7543692600886718,0.7560629580618893,0.7497525900289540,0.7460994050850525,0.7149674549855229
1998,0.7670811844010135,0.7743776578899747,0.7689798792073833,0.7694930781758957,0.7694591476655809,0.7743705890336591,0.7740256288454577,0.7744893458197613,0.7761491132826637,0.7684341634998191,...,0.7718894204668838,0.7668606360839668,0.7709485556912776,0.7663297649746652,0.7562615929243576,0.7527837156170829,0.7479698244661601,0.7512907731632283,0.7430173837314513,0.7212248065960911


In [195]:
driver.close()
driver.quit()

In [196]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
rf_0: [0.76565589 0.77777315]
rf_1: [0.76711377 0.77828391]
rf_2: [0.76530969 0.77789682]
rf_3: [0.76331621 0.77617919]
rf_4: [0.76584675 0.77749686]
rf_5: [0.76615372 0.77767634]
rf_6: [0.76547047 0.77775039]
rf_7: [0.76497428 0.77783606]
rf_8: [0.7655734 0.77764665]
rf_9: [0.76745576 0.77865767]
rf_10: [0.76618326 0.7785287]
rf_11: [0.76652836 0.77891265]
rf_12: [0.76585643 0.77783168]
rf_13: [0.76498311 0.77734573]
rf_14: [0.7658271 0.77803215]
rf_15: [0.76417323 0.77701173]
rf_16: [0.76635539 0.77840341]
rf_17: [0.7642523 0.77668006]
rf_18: [0.76554996 0.77679962]
rf_19: [0.76547609 0.77784472]
rf_20: [0.76759855 0.77906671]
rf_21: [0.76690358 0.77852994]
rf_22: [0.76533026 0.7776774]
rf_23: [0.76744756 0.77880492]
rf_24: [0.76690874 0.77897924]
rf_25: [0.76612689 0.77852602]
rf_26: [0.76612523 0.77837446]
rf_27: [0.76583297 0.7775807]
rf_28: [0.76476914 0.77690248]
rf_29: [0.76485732 0.77590891]
rf_30: [0.76606143 0.77764672]
rf_31: [0.76479897 0.77585508]
rf_32: [0.7670

In [197]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
rf_0: [0.77 0.78]
rf_1: [0.77 0.78]
rf_2: [0.77 0.78]
rf_3: [0.76 0.78]
rf_4: [0.77 0.78]
rf_5: [0.77 0.78]
rf_6: [0.77 0.78]
rf_7: [0.76 0.78]
rf_8: [0.77 0.78]
rf_9: [0.77 0.78]
rf_10: [0.77 0.78]
rf_11: [0.77 0.78]
rf_12: [0.77 0.78]
rf_13: [0.76 0.78]
rf_14: [0.77 0.78]
rf_15: [0.76 0.78]
rf_16: [0.77 0.78]
rf_17: [0.76 0.78]
rf_18: [0.77 0.78]
rf_19: [0.77 0.78]
rf_20: [0.77 0.78]
rf_21: [0.77 0.78]
rf_22: [0.77 0.78]
rf_23: [0.77 0.78]
rf_24: [0.77 0.78]
rf_25: [0.77 0.78]
rf_26: [0.77 0.78]
rf_27: [0.77 0.78]
rf_28: [0.76 0.78]
rf_29: [0.76 0.78]
rf_30: [0.77 0.78]
rf_31: [0.76 0.78]
rf_32: [0.77 0.78]
rf_33: [0.77 0.78]
rf_34: [0.77 0.78]
rf_35: [0.77 0.78]
rf_36: [0.76 0.78]
rf_37: [0.77 0.78]
rf_38: [0.77 0.78]
rf_39: [0.77 0.78]
rf_40: [0.77 0.78]
rf_41: [0.77 0.78]
rf_42: [0.77 0.78]
rf_43: [0.77 0.78]
rf_44: [0.76 0.78]
rf_45: [0.76 0.78]
rf_46: [0.77 0.78]
rf_47: [0.77 0.78]
rf_48: [0.76 0.78]
rf_49: [0.76 0.77]
rf_50: [0.75 0.77]
rf_51: [0.75 0.76]
rf_52: [0.75

In [198]:
i = 0
while i < 55:  
    base_column = f'rf_{i}'
    continue_search = True  

    for j in range(i + 1, 56):
        if not continue_search:
            break  
        compare_column = f'rf_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'rf_55':
            break
    if compare_column == 'rf_55' and continue_search:
        break
    i += 1   

rf_0 and rf_1:
(-0.0010347381635546105, -0.0006639136641687194)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1     -0.0008   9.46e-05     -8.981      0.000      -0.001      -0.001
-------------------------------------------
rf_1 and rf_2:
(0.0006618621806092607, 0.0010388480980735326)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0009   9.61e-05      8.845      0.000       0.001       0.001
-------------------------------------------
rf_1 and rf_3:
(0.0026007255683944464, 0.0029862524672552237)
                          Test for equality of means                          
             

In [199]:
## rf = 20

In [200]:
del new_df

### dt

In [201]:
driver = webdriver.Chrome(options=options)
driver.get(url='.html')

In [202]:
df = pd.DataFrame()

In [203]:
for i in range(0, 775):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

df = df.dropna(axis=1, how='all')

In [204]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [205]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [206]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [207]:
df

,Column_28,Column_44,Column_58,Column_71,Column_84,Column_96,Column_109,Column_122,Column_135,Column_148,...,Column_630,Column_643,Column_656,Column_669,Column_682,Column_695,Column_708,Column_721,Column_734,Column_747
0,"0.733295585640608, 0.7356417390517553, 0.72683...","0.740730608713355, 0.7328057138979371, 0.72577...","0.7458329092019544, 0.7375192272891786, 0.7336...","0.7475732616268549, 0.7311346362649295, 0.7306...","0.7062204521806007, 0.7210897914404633, 0.7154...","0.7122028252804923, 0.7122431177614912, 0.7233...","0.7148246640879479, 0.7053474484256242, 0.7103...","0.7097251911418748, 0.7232804300126674, 0.7111...","0.7305691277596815, 0.7093321627307274, 0.7282...","0.7213633561798769, 0.7183873676710097, 0.7190...",...,"0.7444035864549403, 0.7421677072023164, 0.7388...","0.7447110817046688, 0.7336857865092291, 0.7379...","0.7433460855501266, 0.7327823866720955, 0.7339...","0.7375976915942816, 0.7456173090843287, 0.7362...","0.7457494966974303, 0.7392673554560261, 0.7433...","0.7335415818403909, 0.7423062567861021, 0.7396...","0.7516724914042708, 0.7425791146398841, 0.7472...","0.748796880655085, 0.7413774090662323, 0.73833...","0.7418474880112197, 0.7461538352786825, 0.7451...","0.7182007498642778, 0.7186715356948967, 0.7142..."


In [208]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'dt_{i}' for i in range(len(df.columns))])

In [209]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data) # 2000 * 56

112000

In [210]:
new_df.shape

(2000, 56)

In [211]:
new_df = new_df.astype(float)

In [212]:
new_df

,dt_0,dt_1,dt_2,dt_3,dt_4,dt_5,dt_6,dt_7,dt_8,dt_9,...,dt_46,dt_47,dt_48,dt_49,dt_50,dt_51,dt_52,dt_53,dt_54,dt_55
0,0.7332955856406080,0.7407306087133551,0.7458329092019544,0.7475732616268549,0.7062204521806007,0.7122028252804923,0.7148246640879479,0.7097251911418748,0.7305691277596815,0.7213633561798769,...,0.7444035864549403,0.7447110817046688,0.7433460855501266,0.7375976915942816,0.7457494966974303,0.7335415818403909,0.7516724914042708,0.7487968806550850,0.7418474880112197,0.7182007498642778
1,0.7356417390517553,0.7328057138979371,0.7375192272891786,0.7311346362649295,0.7210897914404633,0.7122431177614912,0.7053474484256242,0.7232804300126674,0.7093321627307274,0.7183873676710097,...,0.7421677072023164,0.7336857865092291,0.7327823866720955,0.7456173090843287,0.7392673554560261,0.7423062567861021,0.7425791146398841,0.7413774090662323,0.7461538352786825,0.7186715356948967
2,0.7268339440825190,0.7257764431777054,0.7336525628845458,0.7306023513843649,0.7154636886988780,0.7233744458016649,0.7103062511310171,0.7111128076366268,0.7282038884364822,0.7190172027687296,...,0.7388898785287731,0.7379009455302208,0.7339494548498009,0.7362468331523706,0.7433906193449150,0.7396384704125951,0.7472848522891784,0.7383363870792615,0.7451359199692363,0.7142407765562794
3,0.7303853374954759,0.7266847912142599,0.7303026318765834,0.7398095367354325,0.7162384353510676,0.7232309480184581,0.7017084011943537,0.7274383312975027,0.7105430578175896,0.7154665162414044,...,0.7301845819761129,0.7326296993756785,0.7445265845548318,0.7384127307274702,0.7499674832609482,0.7452666938110750,0.7414014431777055,0.7409363124321390,0.7488329318222946,0.7208275368711544
4,0.7298099325913863,0.7353186923181324,0.7213951660332971,0.7405517666485704,0.7210636366720955,0.7147886129207384,0.7109551721407890,0.7230549334962000,0.7159973873507058,0.7231948968512487,...,0.7277528954035468,0.7296006944444444,0.7366038103963083,0.7335613746380746,0.7509337959192907,0.7438331297502714,0.7394560939196526,0.7455091555826999,0.7419634172547955,0.7204302671462178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.7298961726384364,0.7394751798317047,0.7348578028863553,0.7342661396127397,0.7085347957383280,0.7046645969055374,0.7222328255066957,0.7057659247195078,0.7235349088400289,0.7109629478827363,...,0.7360969733984799,0.7287255700325732,0.7303549414133189,0.7447584430419834,0.7401658070937387,0.7415619062160694,0.7539111981994209,0.7434160672276511,0.7501074466159972,0.7186771907799492
1996,0.7260450597176982,0.7303330279587404,0.7378182399113283,0.7358898559084329,0.7226548362287369,0.7200874276149113,0.7029546405627940,0.7216213694353962,0.7160200076909157,0.7165855161961636,...,0.7458413918295330,0.7334115148841838,0.7301520652370611,0.7448369073470865,0.7340781080347449,0.7349680770448788,0.7417350931958017,0.7479040841024248,0.7410345695349257,0.7186941560351067
1997,0.7384742297774157,0.7245415139793703,0.7268162719417300,0.7293766116992400,0.7145489786916395,0.7271513357310894,0.7027892293250090,0.7195897801302931,0.6964548271806008,0.7193558009862469,...,0.7372859550307637,0.7397416757148028,0.7482228895222585,0.7394744729460730,0.7401615657799495,0.7396844179786464,0.7485233159156713,0.7420072441639521,0.7468274972855592,0.7158524757962359
1998,0.7419096939467970,0.7314788895674991,0.7296388662685487,0.7317475061074918,0.7108632770086862,0.7192292684581976,0.7143552920285922,0.7069697509500543,0.7121010337495475,0.7161217992218603,...,0.7324642881378935,0.7377616890608035,0.7329096260857764,0.7388722063879841,0.7490923588490771,0.7345255666395223,0.7461983690734708,0.7351235918838218,0.7453550545150200,0.7158680272801302


In [213]:
driver.close()
driver.quit()

In [214]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
dt_0: [0.71988098 0.74487699]
dt_1: [0.72032915 0.74475899]
dt_2: [0.72139434 0.74496669]
dt_3: [0.7200737 0.74506677]
dt_4: [0.69271735 0.73089827]
dt_5: [0.69244949 0.73070527]
dt_6: [0.69309988 0.73068564]
dt_7: [0.69252755 0.73115719]
dt_8: [0.6923478 0.73006517]
dt_9: [0.69203738 0.73113034]
dt_10: [0.69366395 0.73088742]
dt_11: [0.69260311 0.73208748]
dt_12: [0.6920522 0.73130599]
dt_13: [0.68253218 0.72406497]
dt_14: [0.70333217 0.73737529]
dt_15: [0.69219968 0.7314685]
dt_16: [0.69318431 0.73123084]
dt_17: [0.69376258 0.73159049]
dt_18: [0.69374943 0.73187936]
dt_19: [0.69277575 0.73142941]
dt_20: [0.69420826 0.73254478]
dt_21: [0.72001416 0.74525261]
dt_22: [0.72167704 0.74543246]
dt_23: [0.6947934 0.73363729]
dt_24: [0.72251381 0.74543212]
dt_25: [0.72163816 0.74601681]
dt_26: [0.72204205 0.7455743]
dt_27: [0.72143583 0.74566455]
dt_28: [0.72209378 0.74644599]
dt_29: [0.72246889 0.74522562]
dt_30: [0.72228151 0.74628102]
dt_31: [0.7226477 0.74603708]
dt_32: [0.72259

In [215]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
dt_0: [0.72 0.74]
dt_1: [0.72 0.74]
dt_2: [0.72 0.74]
dt_3: [0.72 0.75]
dt_4: [0.69 0.73]
dt_5: [0.69 0.73]
dt_6: [0.69 0.73]
dt_7: [0.69 0.73]
dt_8: [0.69 0.73]
dt_9: [0.69 0.73]
dt_10: [0.69 0.73]
dt_11: [0.69 0.73]
dt_12: [0.69 0.73]
dt_13: [0.68 0.72]
dt_14: [0.7 0.74]
dt_15: [0.69 0.73]
dt_16: [0.69 0.73]
dt_17: [0.69 0.73]
dt_18: [0.69 0.73]
dt_19: [0.69 0.73]
dt_20: [0.69 0.73]
dt_21: [0.72 0.75]
dt_22: [0.72 0.75]
dt_23: [0.69 0.73]
dt_24: [0.72 0.75]
dt_25: [0.72 0.75]
dt_26: [0.72 0.75]
dt_27: [0.72 0.75]
dt_28: [0.72 0.75]
dt_29: [0.72 0.75]
dt_30: [0.72 0.75]
dt_31: [0.72 0.75]
dt_32: [0.72 0.75]
dt_33: [0.72 0.75]
dt_34: [0.72 0.75]
dt_35: [0.72 0.75]
dt_36: [0.72 0.75]
dt_37: [0.72 0.75]
dt_38: [0.72 0.75]
dt_39: [0.72 0.75]
dt_40: [0.72 0.75]
dt_41: [0.72 0.75]
dt_42: [0.72 0.75]
dt_43: [0.72 0.75]
dt_44: [0.72 0.75]
dt_45: [0.72 0.75]
dt_46: [0.72 0.75]
dt_47: [0.72 0.75]
dt_48: [0.72 0.75]
dt_49: [0.73 0.75]
dt_50: [0.73 0.75]
dt_51: [0.73 0.75]
dt_52: [0.74 

In [216]:
i = 0
while i < 55:  
    base_column = f'dt_{i}'
    continue_search = True  

    for j in range(i + 1, 56):
        if not continue_search:
            break  
        compare_column = f'dt_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'dt_55':
            break
    if compare_column == 'dt_55' and continue_search:
        break
    i += 1  

dt_0 and dt_1:
(-0.0002155677147884443, 0.0005701132721539153)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0002      0.000      0.885      0.376      -0.000       0.001
-------------------------------------------
dt_1 and dt_2:
(-0.0005071799862396077, 0.00026270995253484985)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1     -0.0001      0.000     -0.623      0.534      -0.001       0.000
-------------------------------------------
dt_2 and dt_3:
(-0.0005744293046174203, 0.00019255484185081324)
                          Test for equality of means                          
          

In [217]:
## dt = 52